In [15]:
import pandas as pd
import pymc as pm
import arviz as az
import numpy as np

In [16]:
df = pd.read_csv("../data/hw/foxes.csv")
df

,group,avgfood,groupsize,area,weight
0,1,0.37,2,1.09,5.02
1,1,0.37,2,1.09,2.84
2,2,0.53,2,2.05,5.33
3,2,0.53,2,2.05,6.07
4,3,0.49,2,2.12,5.85
...,...,...,...,...,...
111,29,0.67,4,2.75,4.81
112,29,0.67,4,2.75,3.94
113,30,0.41,3,1.91,3.16
114,30,0.41,3,1.91,2.78


In [17]:
avgfood = df['avgfood']
area = df['area']
groupsize = df['groupsize']
weight = df["weight"]

In [18]:
## 1 

model = pm.Model()

# review hw; intercept might be the trouble?; they standardize data ahead of time

with model:
    a_coef = pm.Normal('a_coef', mu=0, sigma=1)
    f_mean = pm.Deterministic('f_mean', a_coef * area)
    f_observed = pm.Normal('f_observed', mu=f_mean, sigma=1, observed=avgfood)

with model:
    trace_prior = pm.sample_prior_predictive(2000)
    trace = pm.sample(2000)
    posterior_pred = pm.sample_posterior_predictive(trace, var_names=['a_coef'])

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a_coef]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 3 seconds.
The acceptance probability does not match the target. It is 0.6991, but should be close to 0.8. Try to increase the number of tuning steps.


In [19]:
az.summary(trace.posterior["a_coef"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a_coef,0.234,0.028,0.182,0.285,0.0,0.0,3712.0,5680.0,1.0


In [20]:
## 2 

model = pm.Model()

with model:
    a_coef = pm.Normal('a_coef', mu=0, sigma=1)
    f_coef = pm.Normal('f_coef', mu=0, sigma=1) 
    g_coef = pm.Normal('g_coef', mu=0, sigma=1) 

    # Define the backdoor adjustment formula
    f_mean = pm.Deterministic('f_mean', a_coef * area + g_coef * groupsize + f_coef * avgfood)

    weight_observed = pm.Normal('weight_observed', mu=f_mean, sigma=1, observed=weight)

with model:
    trace_prior = pm.sample_prior_predictive(2000)
    trace = pm.sample(2000, tune=1000)

    delta_food = 1.0  
    avgfood_intervention = avgfood + delta_food

    model_avgfood_intervention = pm.Deterministic('f_mean_intervention', a_coef * area + g_coef * groupsize + f_coef * avgfood_intervention)

    weight_posterior = pm.sample_posterior_predictive(trace, var_names=['weight_observed', 'f_mean_intervention'])

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a_coef, f_coef, g_coef]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 15 seconds.


In [21]:
az.summary(trace_prior)

arviz - WARNING - Shape validation failed: input_shape: (1, 2000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a_coef,-0.000,0.998,-1.937,1.832,0.022,0.016,2066.0,1929.0,NaN
f_coef,-0.016,0.991,-1.774,1.936,0.023,0.016,1887.0,1978.0,NaN
g_coef,-0.008,0.986,-1.947,1.773,0.022,0.015,2046.0,2097.0,NaN
f_mean[0],-0.023,2.261,-4.147,4.399,0.050,0.035,2045.0,1969.0,NaN
f_mean[1],-0.023,2.261,-4.147,4.399,0.050,0.035,2045.0,1969.0,NaN
...,...,...,...,...,...,...,...,...,...
f_mean[111],-0.045,4.807,-8.874,9.356,0.106,0.075,2056.0,1854.0,NaN
f_mean[112],-0.045,4.807,-8.874,9.356,0.106,0.075,2056.0,1854.0,NaN
f_mean[113],-0.032,3.514,-6.433,6.853,0.078,0.055,2051.0,1891.0,NaN
f_mean[114],-0.032,3.514,-6.433,6.853,0.078,0.055,2051.0,1891.0,NaN


In [22]:
az.summary(trace.posterior["a_coef"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a_coef,1.272,0.183,0.921,1.603,0.003,0.002,3751.0,3257.0,1.0


In [23]:
az.summary(trace.posterior["f_mean"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
f_mean[0],1.583,0.087,1.422,1.748,0.001,0.001,4091.0,4169.0,1.0
f_mean[1],1.583,0.087,1.422,1.748,0.001,0.001,4091.0,4169.0,1.0
f_mean[2],3.501,0.131,3.258,3.748,0.002,0.001,4384.0,4421.0,1.0
f_mean[3],3.501,0.131,3.258,3.748,0.002,0.001,4384.0,4421.0,1.0
f_mean[4],3.416,0.123,3.185,3.649,0.002,0.001,4454.0,4518.0,1.0
...,...,...,...,...,...,...,...,...,...
f_mean[111],3.587,0.083,3.435,3.745,0.001,0.001,7084.0,6407.0,1.0
f_mean[112],3.587,0.083,3.435,3.745,0.001,0.001,7084.0,6407.0,1.0
f_mean[113],2.093,0.083,1.937,2.244,0.001,0.001,5036.0,5497.0,1.0
f_mean[114],2.093,0.083,1.937,2.244,0.001,0.001,5036.0,5497.0,1.0


In [24]:
# az.summary(weight_posterior.posterior_predictive["f_mean_invervention"])
print(weight_posterior.posterior_predictive["f_mean_intervention"].mean())

<xarray.DataArray 'f_mean_intervention' ()>
array(8.58868394)


In [25]:
weight_original = weight_posterior.posterior_predictive['weight_observed']
weight_intervention = weight_posterior.posterior_predictive['f_mean_intervention']

causal_effect = np.mean(weight_intervention) - np.mean(weight_original)

print(causal_effect)

<xarray.DataArray ()>
array(4.3579411)


In [26]:
## 2 after reviewing HW 

model = pm.Model()

with model:
    f_coef = pm.Normal('f_coef', mu=0, sigma=1) 

    # Define the backdoor adjustment formula
    f_mean = pm.Deterministic('f_mean', f_coef * avgfood)

    weight_observed = pm.Normal('weight_observed', mu=f_mean, sigma=1, observed=weight)

with model:
    trace_prior = pm.sample_prior_predictive(2000)
    trace = pm.sample(2000, tune=1000)

    model_avgfood_intervention = pm.Deterministic('f_mean_intervention', f_coef * avgfood_intervention)

    weight_posterior = pm.sample_posterior_predictive(trace, var_names=['weight_observed', 'f_mean_intervention'])

print(weight_posterior.posterior_predictive["f_mean_intervention"].mean())

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [f_coef]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 3 seconds.


<xarray.DataArray 'f_mean_intervention' ()>
array(9.7186007)


In [39]:
avgfood = (df['avgfood'] - df['avgfood'].mean()) / df['avgfood'].std()
groupsize = (df['groupsize'] - df['groupsize'].mean()) / df['groupsize'].std()
weight = (df['weight'] - df['weight'].mean()) / df['weight'].std()

In [41]:
### 3 
with pm.Model() as model:
    a_coef = pm.Normal('a_coef', mu=0, sigma=0.2)
    f_coef = pm.Normal('f_coef', mu=0, sigma=0.5)
    g_coef = pm.Normal('g_coef', mu=0, sigma=0.5)
    sigma = pm.Exponential('sigma', 1)
    
    w_mean = a_coef + f_coef * avgfood + g_coef * groupsize
    
    weight = pm.Normal('weight', mu=w_mean, sigma=sigma, observed=weight)
    
    trace = pm.sample(2000, tune=1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a_coef, f_coef, g_coef, sigma]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 7 seconds.


In [42]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a_coef,0.001,0.082,-0.152,0.156,0.001,0.001,8154.0,5266.0,1.0
f_coef,0.472,0.185,0.106,0.806,0.003,0.002,4543.0,4725.0,1.0
g_coef,-0.568,0.185,-0.922,-0.229,0.003,0.002,4516.0,4526.0,1.0
sigma,0.964,0.065,0.850,1.090,0.001,0.001,7183.0,5299.0,1.0
